In [1]:
import yaml
import torch
import time


from models import SWEGNN, GCN
from models.swe_gnn_ned import SWEGNN_NED
from data import TemporalGraphDataset

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
with open('config.yaml') as f:
    config = yaml.safe_load(f)

dataset, info = TemporalGraphDataset(node_features=config['node_features'],
                    edge_features=config['edge_features'],
                    **config['dataset_parameters']).load()

In [4]:
print(dataset[0])
print(info)

Data(x=[1268, 6], edge_index=[2, 2612], edge_attr=[2612, 8], y=[1268, 1], pos=[2, 1268])
{'num_static_node_features': 3, 'num_dynamic_node_features': 1, 'num_static_edge_features': 5, 'num_dynamic_edge_features': 1, 'previous_timesteps': 2}


In [5]:
num_train = int(len(dataset) * 0.8) # 80% train, 20% test

train_dataset = dataset[:num_train]
# train_loader = DataLoader(train_dataset) # batch_size=32, shuffle=True

test_dataset = dataset[num_train:]
# test_loader = DataLoader(test_dataset) # batch_size=32, shuffle=True

In [6]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
base_model_params = {
    'static_node_features': info['num_static_node_features'],
    'dynamic_node_features': info['num_dynamic_node_features'],
    'static_edge_features': info['num_static_edge_features'],
    'dynamic_edge_features': info['num_dynamic_edge_features'],
    'previous_timesteps': info['previous_timesteps'],
    'device': device,
}
lr_info = config['training_parameters']

In [7]:
def train(model, loss_func, optimizer):
    start_time = time.time()
    num_epochs = 10
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for graph in train_dataset:
            graph = graph.to(device)
            labels = graph.y

            optimizer.zero_grad()

            outputs = model(graph)
            loss = loss_func(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / num_train
        print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {epoch_loss:.4f}')
    end_time = time.time()
    print(f'Total training time: {end_time - start_time} seconds')


def test(model, loss_func):
    start_time = time.time()
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for graph in test_dataset:
            graph = graph.to(device)
            labels = graph.y

            outputs = model(graph)
            loss = loss_func(outputs, labels)
            running_loss += loss.item()
    end_time = time.time()

    # Print validation statistics
    print(f'Validation Loss: {running_loss:.4f}')
    print(f'Inference time: {end_time - start_time} seconds')

In [8]:
gcn_params = config['GCN']
model = GCN(**gcn_params, **base_model_params)
optimizer = torch.optim.Adam(model.parameters(), lr=lr_info['learning_rate'], weight_decay=lr_info['weight_decay'])
loss_func = torch.nn.L1Loss()

train(model, loss_func, optimizer)

Epoch [1/10], Training Loss: 46.5382
Epoch [2/10], Training Loss: 29.1531
Epoch [3/10], Training Loss: 26.0924
Epoch [4/10], Training Loss: 20.8580
Epoch [5/10], Training Loss: 16.7502
Epoch [6/10], Training Loss: 17.2482
Epoch [7/10], Training Loss: 15.0181
Epoch [8/10], Training Loss: 14.4506
Epoch [9/10], Training Loss: 29.5900
Epoch [10/10], Training Loss: 25.3774
Total training time: 56.29916191101074 seconds


In [9]:
test(model, loss_func)

Validation Loss: 1673.8410
Inference time: 0.8602676391601562 seconds


In [10]:
swe_gnn_params = config['SWEGNN']
model = SWEGNN(**swe_gnn_params, **base_model_params)
optimizer = torch.optim.Adam(model.parameters(), lr=lr_info['learning_rate'], weight_decay=lr_info['weight_decay'])
loss_func = torch.nn.L1Loss()
train(model, loss_func, optimizer)

Epoch [1/10], Training Loss: 0.0665
Epoch [2/10], Training Loss: 0.0206
Epoch [3/10], Training Loss: 0.0206
Epoch [4/10], Training Loss: 0.0206
Epoch [5/10], Training Loss: 0.0893
Epoch [6/10], Training Loss: 0.0332
Epoch [7/10], Training Loss: 0.0238
Epoch [8/10], Training Loss: 0.0323
Epoch [9/10], Training Loss: 0.2593
Epoch [10/10], Training Loss: 0.0410
Total training time: 468.9008831977844 seconds


In [11]:
test(model, loss_func)

Validation Loss: 0.1904
Inference time: 1.2257566452026367 seconds


In [12]:
swe_gnn_params = config['SWEGNN']
if 'hidden_features' in swe_gnn_params:
    del swe_gnn_params['hidden_features']
model = SWEGNN_NED(**swe_gnn_params, **base_model_params)
optimizer = torch.optim.Adam(model.parameters(), lr=lr_info['learning_rate'], weight_decay=lr_info['weight_decay'])
loss_func = torch.nn.L1Loss()
train(model, loss_func, optimizer)

Epoch [1/10], Training Loss: 1.8126
Epoch [2/10], Training Loss: 0.1243
Epoch [3/10], Training Loss: 0.0650
Epoch [4/10], Training Loss: 0.0504
Epoch [5/10], Training Loss: 0.0261
Epoch [6/10], Training Loss: 0.0327
Epoch [7/10], Training Loss: 0.0258
Epoch [8/10], Training Loss: 0.0218
Epoch [9/10], Training Loss: 0.0209
Epoch [10/10], Training Loss: 0.0207
Total training time: 119.28316760063171 seconds


In [13]:
test(model, loss_func)

Validation Loss: 0.1826
Inference time: 1.061037302017212 seconds
